# Train Music Transformer
Since Fri. May 13th, 2022

Set up training in colab


## Setup


### Ipython



In [1]:
%load_ext autoreload
%autoreload 2



### Colab



In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""


import os
import sys

import torch


if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream music21 transformers datasets
    ! pip3 install stefutils

    path = os.path.join('drive', 'My Drive', 'Research', 'Music with NLP', 'Symbolic-Music-Generation')
    sys.path.append(path)
    ! ls "{path}"


    import time, os
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()

    if torch.cuda.is_available():
        %env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

if torch.cuda.is_available():
    ! nvidia-smi


from stefutil import *
from musicnlp.util import *

mic(u.proj_dir, u.pkg_nm)



Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.7 MB/s 
     |████████████████████████████████| 346 kB 93.2 MB/s 
     |████████████████████████████████| 6.6 MB 74.7 MB/s 
     |████████████████████████████████| 596 kB 74.0 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 140 kB 83.8 MB/s 
     |████████████████████████████████| 212 kB 72.7 MB/s 
     |████████████████████████████████| 1.1 MB 59.7 MB/s 
     |████████████████████████████████| 86 kB 7.6 MB/s 
     |████████████████████████████████| 127 kB 82.6 MB/s 
     |████████████████████████████████| 271 kB 70.9 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 144 kB 93.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stefutils: filename=stefutils-0.7.1-py3-none-any.whl size=17997 sha256=0a2679962dd332c57ea8bdd7abacbc18a1f05f14e0532f5a25ee6caf3017489b
  Stored in directory: /root/.cache/pip/wheels/f1/56/5c/f16342f9bfd950674616d45668f6a5272e8b5639c581390b39
Successfully built stefutils
eval  musicnlp	notebook  plot
env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
Wed Jun  1 00:00:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============

ic| u.proj_dir: 'Symbolic-Music-Generation', u.pkg_nm: 'musicnlp'


('Symbolic-Music-Generation', 'musicnlp')

### code

In [ ]:
import transformers

from musicnlp.trainer import train

seed = sconfig('random-seed')
transformers.set_seed(seed)



## Prep for training


In [ ]:
md_nm = 'transf-xl'
# md_sz = 'debug'
# md_sz = 'debug-large'
md_sz = 'base'
# n = 32
n = None
# n = 4096 * 4
mic(md_sz, n)

augment_key = False

# max_length = 512
max_length = 1024
mem_len = 256
# mem_len = 128

gas = 2
model_config = dict(max_length=max_length)   # TODO: short length to test for now
num_train_epochs = 4
# bsz = 16
bsz = 20

pop = 'musicnlp music extraction, dnm=POP909, n=909, meta={mode=melody, prec=5, th=1}, 2022-05-20_14-52-04'
mst = 'musicnlp music extraction, dnm=MAESTRO, n=1276, meta={mode=melody, prec=5, th=1}, 2022-05-20_14-52-28'
lmd = 'musicnlp music extraction, dnm=LMD, n=176640, meta={mode=melody, prec=5, th=1}, 2022-05-27_15-23-20'
dnms = [pop, mst, lmd]
my_train_args = dict(
    augment_key=augment_key,
    save_epochs=2
)
train_args = dict()

with_tqdm = True
if with_tqdm:
    my_train_args.update(dict(tqdm=True, logging_strategy='epoch'))
if 'debug' not in md_sz:
    model_config.update(mem_len=mem_len)  # TODO
    train_args = dict(
        gradient_accumulation_steps=gas,
        per_device_train_batch_size=bsz,
        per_device_eval_batch_size=bsz,
        num_train_epochs=num_train_epochs
    )
mdl, tokenizer, trainer = train.get_all_setup(
    model_name=md_nm, model_size=md_sz, model_config=model_config,
    dataset_names=dnms, n_sample=n, dataset_args=dict(shuffle_seed=seed),
    train_args=train_args, my_train_args=my_train_args
)
trainer.train_dataset = trainer.train_dataset.select(range(8))  # TODO: debugging eval, remove



## Train


Check log and tensorboard files written



In [ ]:
ignore_keys_for_eval = ['losses', 'mems', 'hidden_states', 'attentions']
# trainer.train(ignore_keys_for_eval=ignore_keys_for_eval)
resume = os.path.join(u.model_path, 'checkpoint-17526')
trainer.train(resume_from_checkpoint=resume, ignore_keys_for_eval=ignore_keys_for_eval)
trainer.save_model(os.path.join(trainer.args.output_dir, 'trained'))



2022-06-01 00:02:45|[MyTransfoXLLMHeadModel Train]::on_train_begin::train_util_wrap.py:170:INFO: Training started with model {model name: MyTransfoXLLMHeadModel, max length: 512, n_layer: 12, hidden_size: 768, ff_size: 3072, parameter_count: 92.4M}, {
    "vocab_size": 418,
    "cutoffs": [],
    "tie_projs": [
        false
    ],
    "d_model": 768,
    "d_embed": 768,
    "d_head": 64,
    "d_inner": 3072,
    "div_val": 1,
    "pre_lnorm": false,
    "n_layer": 12,
    "n_head": 12,
    "mem_len": 256,
    "same_length": true,
    "attn_type": 0,
    "clamp_len": 1000,
    "sample_softmax": -1,
    "adaptive": true,
    "dropout": 0.1,
    "dropatt": 0.0,
    "untie_r": true,
    "init": "normal",
    "init_range": 0.01,
    "proj_init_std": 0.01,
    "init_std": 0.02,
    "layer_norm_epsilon": 1e-05,
    "return_dict": true,
    "output_hidden_states": false,
    "output_attentions": false,
    "torchscript": false,
    "torch_dtype": null,
    "use_bfloat16": false,
    "pruned_h

Train Epoch 1/8:   0%|          | 0/512 [00:00<?, ?ba/s]

2022-06-01 00:52:25|[MyTransfoXLLMHeadModel Train]::_log::train_util_wrap.py:252:INFO: {step:  512/4096, epoch: 1.000/8, train/learning_rate: 2.994e-04, train/loss: 2.998, train/ntp_acc: 38.65, train/ikr: 80.25}


Eval Epoch 2/8:   0%|          | 0/224 [00:00<?, ?ba/s]

2022-06-01 00:59:04|[MyTransfoXLLMHeadModel Train]::_log::train_util_wrap.py:252:INFO: {step:  512/4096, epoch: 1/8, eval/loss: 3.3111, eval/ntp_acc: 39.61, eval/ikr: 84.27}


...

Train Epoch 4/8:   0%|          | 0/512 [00:00<?, ?ba/s]

2022-06-01 03:48:20|[MyTransfoXLLMHeadModel Train]::_log::train_util_wrap.py:252:INFO: {step: 2048/4096, epoch: 4.000/8, train/learning_rate: 1.761e-04, train/loss: 3.4222, train/ntp_acc: 53.76, train/ikr:  82.5}


Eval Epoch 5/8:   0%|          | 0/224 [00:00<?, ?ba/s]

2022-06-01 03:55:23|[MyTransfoXLLMHeadModel Train]::_log::train_util_wrap.py:252:INFO: {step: 2048/4096, epoch: 4/8, eval/loss: 3.173, eval/ntp_acc: 54.91, eval/ikr: 84.09}


Train Epoch 5/8:   0%|          | 0/512 [00:00<?, ?ba/s]

## Sanity check



In [ ]:
mic(trainer.args.output_dir)
mic(os.listdir(trainer.args.output_dir))

